In [3]:
# Cell 1: Import an# Cell 1: Import and check modules
import sys
import os
import numpy as np
import openmdao.api as om

# Add FELIN to path if needed
sys.path.append('.')  # Adjust if your notebook is in a subdirectory

# Check imports
try:
    import Launch_vehicle_Group
    print("✓ Launch_vehicle_Group imported")
except ImportError as e:
    print(f"✗ Error: {e}")
    
try:
    import post_traitement
    import constants as Cst
    import specifications as Spec
    print("✓ FELIN modules found")
except ImportError as e:
    print(f"✗ Error: {e}")
import sys
import os
import numpy as np
import openmdao.api as om

# Add FELIN to path if needed
sys.path.append('.')  # Adjust if your notebook is in a subdirectory

# Check imports
try:
    import Launch_vehicle_Group
    print("✓ Launch_vehicle_Group imported")
except ImportError as e:
    print(f"✗ Error: {e}")
    
try:
    import post_traitement
    import constants as Cst
    import specifications as Spec
    print("✓ FELIN modules found")
except ImportError as e:
    print(f"✗ Error: {e}")

✓ Environmental discipline loaded
✓ Launch_vehicle_Group imported
✓ FELIN modules found
✓ Launch_vehicle_Group imported
✓ FELIN modules found


In [4]:
# Cell 2: Test FELIN WITHOUT environmental discipline
Launch_vehicle_Group.lca_available = False  # Disable environmental

P_test = om.Problem()
P_test.model = Launch_vehicle_Group.Launcher_vehicle()
P_test.setup()

# Set some basic values
P_test['Prop_mass_stage_1'] = 320000.
P_test['Prop_mass_stage_2'] = 75000.
P_test['OF_stage_1'] = 5.0
P_test['OF_stage_2'] = 5.5

# Run
P_test.run_model()

# Check results
print(f"GLOW: {P_test['GLOW'][0]/1e3:.1f} t")
print(f"Dry mass stage 1: {P_test['Dry_mass_stage_1'][0]/1e3:.1f} t")
print(f"Dry mass stage 2: {P_test['Dry_mass_stage_2'][0]/1e3:.1f} t")

⚠ Running without environmental discipline
GLOW: 438.9 t
Dry mass stage 1: 30.0 t
Dry mass stage 2: 6.9 t


In [5]:
# Cell 3: Test environmental discipline standalone
from Environmental_Discipline import FELINEnvironmentalDiscipline

# Create a Group to wrap the component
env_group = om.Group()
env_group.add_subsystem('env_disc', FELINEnvironmentalDiscipline(), promotes=['*'])

env = om.Problem()
env.model = env_group
env.setup()

# Use values from previous FELIN run
env['Dry_mass_stage_1'] = P_test['Dry_mass_stage_1'][0]
env['Dry_mass_stage_2'] = P_test['Dry_mass_stage_2'][0]
env['Prop_mass_stage_1'] = P_test['Prop_mass_stage_1'][0]
env['Prop_mass_stage_2'] = P_test['Prop_mass_stage_2'][0]
env['GLOW'] = P_test['GLOW'][0]

# Trajectory arrays
env['V_ascent'] = P_test['V_ascent']
env['nx_ascent'] = P_test['nx_ascent']
env['alt_ascent'] = P_test['alt_ascent']
env['Nb_pt_ascent'] = P_test['Nb_pt_ascent'][0]
env['max_pdyn_load_ascent_stage_1'] = P_test['max_pdyn_load_ascent_stage_1'][0]

# Propulsion parameters
env['N_eng_stage_1'] = P_test['N_eng_stage_1'][0]
env['N_eng_stage_2'] = P_test['N_eng_stage_2'][0]
env['OF_stage_1'] = P_test['OF_stage_1'][0]
env['OF_stage_2'] = P_test['OF_stage_2'][0]
env['Thrust_stage_1'] = P_test['Thrust_stage_1'][0]
env['Thrust_stage_2'] = P_test['Thrust_stage_2'][0]
env['Isp_stage_1'] = P_test['Isp_stage_1'][0]
env['Isp_stage_2'] = P_test['Isp_stage_2'][0]
env['payload_mass'] = 5000.0

# Material fractions
env['cfrp_fraction_stage1'] = 0.25
env['aluminum_fraction_stage1'] = 0.65
env['steel_fraction_stage1'] = 0.10
env['cfrp_fraction_stage2'] = 0.35
env['aluminum_fraction_stage2'] = 0.55
env['steel_fraction_stage2'] = 0.10
env['engine_nickel_fraction'] = 0.60
env['engine_steel_fraction'] = 0.30
env['engine_titanium_fraction'] = 0.10

# Run
env.run_model()

print(f"GWP total: {env['GWP_total'][0]/1e3:.1f} tCO2eq")
print(f"ESA single score: {env['ESA_single_score'][0]:.6f}")
print(f"Delta-V achieved: {env['delta_v_achieved'][0]:.1f} m/s")
print(f"Max acceleration: {env['max_acceleration_g'][0]:.1f} g")

# Check some material masses
print(f"\nMaterial breakdown:")
print(f"Stage 1 CFRP: {env['stage1_cfrp_mass'][0]:.0f} kg")
print(f"Stage 1 Aluminum: {env['stage1_aluminum_mass'][0]:.0f} kg")
print(f"Total LOX: {env['total_lox_mass'][0]/1e3:.1f} t")
print(f"Total LH2: {env['total_lh2_mass'][0]/1e3:.1f} t")

GWP total: 1188.7 tCO2eq
ESA single score: 7856.880018
Delta-V achieved: 831.8 m/s
Max acceleration: 1.6 g

Material breakdown:
Stage 1 CFRP: 7495 kg
Stage 1 Aluminum: 19487 kg
Total LOX: 330.1 t
Total LH2: 64.9 t


In [6]:
# Cell 3b: Debug trajectory values
nb_pts = int(P_test['Nb_pt_ascent'][0])
print(f"Number of trajectory points: {nb_pts}")
print(f"Initial velocity: {P_test['V_ascent'][0]:.1f} m/s")
print(f"Final velocity: {P_test['V_ascent'][nb_pts-1]:.1f} m/s")
print(f"Max nx during ascent: {np.max(P_test['nx_ascent'][:nb_pts]):.2f}")
print(f"Final altitude: {P_test['alt_ascent'][nb_pts-1]/1000:.1f} km")

Number of trajectory points: 45
Initial velocity: 10.0 m/s
Final velocity: 841.8 m/s
Max nx during ascent: 1.59
Final altitude: 0.0 km


In [7]:
# Cell 3c: Check trajectory failure
print("Trajectory diagnostic:")
print(f"Time of simulation: {P_test['T_ascent'][44]:.1f} s")
print(f"Stage 1 burn time expected: {320000/(7*219*0.8):.1f} s")
print(f"Max altitude reached: {np.max(P_test['alt_ascent'][:45])/1000:.1f} km")

# Check propellant vs dry mass ratio
print(f"\nMass ratios:")
print(f"Stage 1: Prop/Dry = {320000/30000:.1f}")
print(f"Stage 2: Prop/Dry = {75000/6900:.1f}")

Trajectory diagnostic:
Time of simulation: 87.3 s
Stage 1 burn time expected: 260.9 s
Max altitude reached: 4.0 km

Mass ratios:
Stage 1: Prop/Dry = 10.7
Stage 2: Prop/Dry = 10.9


In [8]:
# Cell 3d: Check ESA category values
print("ESA Categories (raw values):")
for code in ['GWP', 'ODEPL', 'PMAT', 'LUP']:
    raw = env[f'ESA_{code}'][0]
    norm = env[f'ESA_{code}_normalized'][0]
    print(f"{code}: raw={raw:.2e}, normalized={norm:.2e}")

ESA Categories (raw values):
GWP: raw=1.19e+06, normalized=1.47e+02
ODEPL: raw=2.48e-01, normalized=4.63e+00
PMAT: raw=5.12e+01, normalized=8.61e+04
LUP: raw=1.54e+08, normalized=1.88e+02


In [9]:
# Cell 4: Test with more conservative parameters
Launch_vehicle_Group.lca_available = False

P_conservative = om.Problem()
P_conservative.model = Launch_vehicle_Group.Launcher_vehicle()
P_conservative.setup()

# More conservative values
P_conservative['Prop_mass_stage_1'] = 250000.  # Less propellant
P_conservative['Prop_mass_stage_2'] = 50000.
P_conservative['OF_stage_1'] = 2.7  # Lower O/F ratio
P_conservative['OF_stage_2'] = 2.7
P_conservative['N_eng_stage_1'] = 7.
P_conservative['payload_mass'] = 5000.

# Run
P_conservative.run_model()

nb_pts_cons = int(P_conservative['Nb_pt_ascent'][0])
print(f"Points: {nb_pts_cons}")
print(f"Final altitude: {P_conservative['alt_ascent'][nb_pts_cons-1]/1000:.1f} km")
print(f"Final velocity: {P_conservative['V_ascent'][nb_pts_cons-1]:.1f} m/s")
print(f"GLOW: {P_conservative['GLOW'][0]/1e3:.1f} t")

⚠ Running without environmental discipline
Points: 110
Final altitude: 0.0 km
Final velocity: 1525.6 m/s
GLOW: 343.7 t


In [10]:
# Cell 5: Use FELIN default parameters that should work
P_default = om.Problem()
P_default.model = Launch_vehicle_Group.Launcher_vehicle()
P_default.setup()

# Use original FELIN defaults
P_default['Diameter_stage_1'] = 4.6
P_default['Diameter_stage_2'] = 4.6
P_default['Thrust_stage_1'] = 1000.
P_default['Thrust_stage_2'] = 1150.
P_default['OF_stage_1'] = 2.7  # Original FELIN value
P_default['OF_stage_2'] = 2.7  # Original FELIN value
P_default['Prop_mass_stage_1'] = 350000.
P_default['Prop_mass_stage_2'] = 75000.
P_default['N_eng_stage_1'] = 7.
P_default['payload_mass'] = 5000.

# Run
P_default.run_model()

nb_pts = int(P_default['Nb_pt_ascent'][0])
print(f"Trajectory points: {nb_pts}")
print(f"Simulation time: {P_default['T_ascent'][nb_pts-1]:.1f} s")
print(f"Final altitude: {P_default['alt_ascent'][nb_pts-1]/1000:.1f} km")
print(f"Final velocity: {P_default['V_ascent'][nb_pts-1]:.1f} m/s")
print(f"Max altitude reached: {np.max(P_default['alt_ascent'][:nb_pts])/1000:.1f} km")
print(f"GLOW: {P_default['GLOW'][0]/1e3:.1f} t")

⚠ Running without environmental discipline
Trajectory points: 21
Simulation time: 39.3 s
Final altitude: 0.0 km
Final velocity: 251.0 m/s
Max altitude reached: 0.7 km
GLOW: 473.7 t


In [11]:
# Cell 6: Run with proper FPI convergence
def FPI_test(Pb):
    """Fixed Point Iteration from original FELIN"""
    Pb.setup(check=False)
    
    # FELIN baseline configuration
    Pb['Diameter_stage_1'] = 4.6
    Pb['Diameter_stage_2'] = 4.6
    Pb['Mass_flow_rate_stage_1'] = 219.  # Original FELIN value
    Pb['Mass_flow_rate_stage_2'] = 219.
    Pb['Thrust_stage_1'] = 1000.
    Pb['Thrust_stage_2'] = 1150.
    Pb['OF_stage_1'] = 2.7  # LOX/LH2 typical
    Pb['OF_stage_2'] = 2.7
    Pb['Pc_stage_1'] = 100.0
    Pb['Pc_stage_2'] = 100.0
    Pb['Pe_stage_1'] = 1.0
    Pb['Pe_stage_2'] = 1.0
    Pb['N_eng_stage_1'] = 7.
    Pb['N_eng_stage_2'] = 1.
    Pb['Prop_mass_stage_1'] = 350000.
    Pb['Prop_mass_stage_2'] = 75000.
    Pb['thetacmd_i'] = 2.72
    Pb['thetacmd_f'] = 20.
    Pb['ksi'] = 0.293
    Pb['Pitch_over_duration'] = 5.
    Pb['Exit_nozzle_area_stage_1'] = 0.82  # From FELIN
    Pb['Exit_nozzle_area_stage_2'] = 1.0
    Pb['Delta_vertical_phase'] = 10.
    Pb['Delta_theta_pitch_over'] = 1.
    Pb['command_stage_1_exo'] = np.array([1., 1.])
    Pb['is_fallout'] = 0.  # No fallout simulation
    Pb['payload_mass'] = 5000.0
    
    # CRITICAL: Fixed point iteration for pdyn convergence
    error = 100.
    Pb['Pdyn_max_dim'] = 40.
    k = 0
    
    while error > 1. and k < 20:
        Pb.run_model()
        error = abs(Pb['Pdyn_max_dim'] - Pb['max_pdyn_load_ascent_stage_1']/1e3)
        Pb['Pdyn_max_dim'] = Pb['max_pdyn_load_ascent_stage_1']/1e3
        k = k + 1
        print(f'FPI iteration {k}: error = {error[0]:.2f}, Pdyn_max = {Pb["Pdyn_max_dim"][0]:.2f} kPa')
    
    return Pb

# Create new problem and run with FPI
P_fpi = om.Problem()
P_fpi.model = Launch_vehicle_Group.Launcher_vehicle()
P_fpi = FPI_test(P_fpi)

# Check results
nb_pts = int(P_fpi['Nb_pt_ascent'][0])
print(f"\nFinal results:")
print(f"Trajectory points: {nb_pts}")
print(f"Final altitude: {P_fpi['alt_ascent'][nb_pts-1]/1000:.1f} km")
print(f"Final velocity: {P_fpi['V_ascent'][nb_pts-1]:.1f} m/s")
print(f"GLOW: {P_fpi['GLOW'][0]/1e3:.1f} t")

⚠ Running without environmental discipline
FPI iteration 1: error = 22.41, Pdyn_max = 17.59 kPa
FPI iteration 2: error = 0.67, Pdyn_max = 18.26 kPa

Final results:
Trajectory points: 17
Final altitude: 0.0 km
Final velocity: 172.7 m/s
GLOW: 472.7 t


In [12]:
# Cell 7: Check what parameters actually work in original FELIN
# Run the exact baseline from Launcher_Design_Problem_PYTHON.py

P_original = om.Problem()
P_original.model = Launch_vehicle_Group.Launcher_vehicle()
P_original.setup(check=False)

# EXACT values from original FELIN FPI function
P_original['Diameter_stage_1'] = 5.0  # Not 4.6
P_original['Diameter_stage_2'] = 5.0
P_original['Mass_flow_rate_stage_1'] = 250.  # Not 219
P_original['Mass_flow_rate_stage_2'] = 250.
P_original['Thrust_stage_1'] = 1000.
P_original['Thrust_stage_2'] = 800.  # Not 1150
P_original['OF_stage_1'] = 5.0  # Not 2.7
P_original['OF_stage_2'] = 5.0
P_original['Pc_stage_1'] = 100.0
P_original['Pc_stage_2'] = 100.0
P_original['Pe_stage_1'] = 1.0
P_original['Pe_stage_2'] = 1.0
P_original['N_eng_stage_1'] = 8.  # Not 7
P_original['N_eng_stage_2'] = 1.
P_original['Prop_mass_stage_1'] = 320000.
P_original['Prop_mass_stage_2'] = 75000.
P_original['thetacmd_i'] = 2.72
P_original['thetacmd_f'] = 10.  # Not 20
P_original['ksi'] = 0.293
P_original['Pitch_over_duration'] = 5.
P_original['Exit_nozzle_area_stage_1'] = 0.79  # Not 0.82
P_original['Exit_nozzle_area_stage_2'] = 3.6305  # Not 1.0
P_original['Delta_vertical_phase'] = 10.
P_original['Delta_theta_pitch_over'] = 1.
P_original['command_stage_1_exo'] = np.array([30., -20.])  # Not [1., 1.]
P_original['is_fallout'] = 0.
P_original['payload_mass'] = 5000.0

# FPI
error = 100.
P_original['Pdyn_max_dim'] = 40.
k = 0

while error > 1. and k < 20:
    P_original.run_model()
    error = abs(P_original['Pdyn_max_dim'] - P_original['max_pdyn_load_ascent_stage_1']/1e3)
    P_original['Pdyn_max_dim'] = P_original['max_pdyn_load_ascent_stage_1']/1e3
    k = k + 1
    print(f'Iteration {k}: error = {error[0]:.2f}, Pdyn = {P_original["Pdyn_max_dim"][0]:.2f} kPa')

nb_pts = int(P_original['Nb_pt_ascent'][0])
print(f"\nResults:")
print(f"Points: {nb_pts}")
print(f"Final altitude: {P_original['alt_ascent'][nb_pts-1]/1000:.1f} km")
print(f"Final velocity: {P_original['V_ascent'][nb_pts-1]:.1f} m/s")
print(f"Max altitude: {np.max(P_original['alt_ascent'][:nb_pts])/1000:.1f} km")
print(f"GLOW: {P_original['GLOW'][0]/1e3:.1f} t")

⚠ Running without environmental discipline
Iteration 1: error = 3.77, Pdyn = 36.23 kPa
Iteration 2: error = 0.01, Pdyn = 36.24 kPa

Results:
Points: 101
Final altitude: 0.0 km
Final velocity: 1934.9 m/s
Max altitude: 76.3 km
GLOW: 441.2 t


In [13]:
# Cell 8: Test environmental with working trajectory data
from Environmental_Discipline import FELINEnvironmentalDiscipline

env_group = om.Group()
env_group.add_subsystem('env_disc', FELINEnvironmentalDiscipline(), promotes=['*'])

env = om.Problem()
env.model = env_group
env.setup()

# Use values from working FELIN run
env['Dry_mass_stage_1'] = P_original['Dry_mass_stage_1'][0]
env['Dry_mass_stage_2'] = P_original['Dry_mass_stage_2'][0]
env['Prop_mass_stage_1'] = P_original['Prop_mass_stage_1'][0]
env['Prop_mass_stage_2'] = P_original['Prop_mass_stage_2'][0]
env['GLOW'] = P_original['GLOW'][0]
env['V_ascent'] = P_original['V_ascent']
env['nx_ascent'] = P_original['nx_ascent']
env['alt_ascent'] = P_original['alt_ascent']
env['Nb_pt_ascent'] = P_original['Nb_pt_ascent'][0]
env['max_pdyn_load_ascent_stage_1'] = P_original['max_pdyn_load_ascent_stage_1'][0]

# Propulsion parameters
env['N_eng_stage_1'] = P_original['N_eng_stage_1'][0]
env['N_eng_stage_2'] = P_original['N_eng_stage_2'][0]
env['OF_stage_1'] = P_original['OF_stage_1'][0]  # 5.0
env['OF_stage_2'] = P_original['OF_stage_2'][0]  # 5.0
env['Thrust_stage_1'] = P_original['Thrust_stage_1'][0]
env['Thrust_stage_2'] = P_original['Thrust_stage_2'][0]
env['Isp_stage_1'] = P_original['Isp_stage_1'][0]
env['Isp_stage_2'] = P_original['Isp_stage_2'][0]
env['payload_mass'] = 5000.0

# Material fractions
env['cfrp_fraction_stage1'] = 0.25
env['aluminum_fraction_stage1'] = 0.65
env['steel_fraction_stage1'] = 0.10
env['cfrp_fraction_stage2'] = 0.35
env['aluminum_fraction_stage2'] = 0.55
env['steel_fraction_stage2'] = 0.10
env['engine_nickel_fraction'] = 0.60
env['engine_steel_fraction'] = 0.30
env['engine_titanium_fraction'] = 0.10

env.run_model()

print(f"GWP total: {env['GWP_total'][0]/1e3:.1f} tCO2eq")
print(f"ESA single score: {env['ESA_single_score'][0]:.2e}")
print(f"Delta-V: {env['delta_v_achieved'][0]:.1f} m/s")
print(f"Max acceleration: {env['max_acceleration_g'][0]:.1f} g")

GWP total: 1232.8 tCO2eq
ESA single score: 8.15e+03
Delta-V: 1924.9 m/s
Max acceleration: 7.4 g


In [15]:
# Cell 9: Run partial optimization to reach orbit
import openmdao.api as om  # Add this import!
import cma
import numpy as np
import Launch_vehicle_Group

def objective_simple(x):
    """Simple objective to reach orbital velocity"""
    P = om.Problem()
    P.model = Launch_vehicle_Group.Launcher_vehicle()
    P.setup(check=False)
    
    # Design variables
    P['Prop_mass_stage_1'] = x[0] * 1e3
    P['Prop_mass_stage_2'] = x[1] * 1e3
    
    # Fixed good values
    P['Diameter_stage_1'] = 5.0
    P['Diameter_stage_2'] = 5.0
    P['Mass_flow_rate_stage_1'] = 250.
    P['Mass_flow_rate_stage_2'] = 250.
    P['Thrust_stage_1'] = 1000.
    P['Thrust_stage_2'] = 800.
    P['OF_stage_1'] = 5.0
    P['OF_stage_2'] = 5.0
    P['N_eng_stage_1'] = 8.
    P['N_eng_stage_2'] = 1.
    P['payload_mass'] = 5000.
    
    # FPI
    error = 100.
    P['Pdyn_max_dim'] = 40.
    k = 0
    while error > 1. and k < 10:
        try:
            P.run_model()
            error = abs(P['Pdyn_max_dim'] - P['max_pdyn_load_ascent_stage_1']/1e3)
            P['Pdyn_max_dim'] = P['max_pdyn_load_ascent_stage_1']/1e3
            k += 1
        except:
            return 1e6
    
    # Get final velocity
    nb_pts = int(P['Nb_pt_ascent'][0])
    v_final = P['V_ascent'][nb_pts-1]
    
    # Objective: reach 7800 m/s orbital velocity
    return abs(v_final - 7800) / 100

# Quick optimization
x0 = [400, 100]  # Initial guess: 400t and 100t propellant
es = cma.CMAEvolutionStrategy(x0, 50, {'bounds': [[250, 50], [500, 150]], 'maxiter': 20})
es.optimize(objective_simple)

print(f"Best propellant masses: {es.result.xbest}")

(3_w,6)-aCMA-ES (mu_w=2.0,w_1=63%) in dimension 2 (seed=481795, Thu Aug 28 11:32:03 2025)
⚠ Running without environmental discipline
⚠ Running without environmental discipline
⚠ Running without environmental discipline
⚠ Running without environmental discipline
⚠ Running without environmental discipline
⚠ Running without environmental discipline
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      6 2.323863764380130e+00 1.0e+00 3.77e+01  3e+01  4e+01 0:02.1
⚠ Running without environmental discipline
⚠ Running without environmental discipline
⚠ Running without environmental discipline
⚠ Running without environmental discipline
⚠ Running without environmental discipline
⚠ Running without environmental discipline
    2     12 5.640281193720245e+01 1.1e+00 3.02e+01  2e+01  2e+01 0:03.8
⚠ Running without environmental discipline
⚠ Running without environmental discipline
⚠ Running without environmental discipline
⚠ Running without environmental discipline
⚠ Ru

In [16]:
# Cell 10: Proper Brightway2 ESA calculation
import brightway2 as bw

# Check if project exists
if "LCA_FELIN" not in bw.projects:
    print("Creating LCA_FELIN project...")
    bw.projects.create_project("LCA_FELIN")
    
bw.projects.set_current("LCA_FELIN")

# Check available databases
print("Available databases:", list(bw.databases))

# Function to calculate all ESA categories
def calculate_ESA_score(material_masses):
    """
    material_masses: dict with keys like 'aluminum', 'cfrp', 'steel', etc.
    """
    
    # ESA impact categories and their LCIA methods
    esa_methods = {
        'GWP': ('IPCC 2013', 'climate change', 'GWP 100a'),
        'ODEPL': ('CML 2001', 'ozone depletion', 'ODP 40a'),
        # Add other methods...
    }
    
    # Your normalization and weights
    ESA_NORM = {
        'GWP': 0.0001235,
        'ODEPL': 18.64,
        # ... from your table
    }
    
    ESA_WEIGHTS = {
        'GWP': 0.2106,
        'ODEPL': 0.0631,
        # ... from your table
    }
    
    total_score = 0.0
    
    for category, method_tuple in esa_methods.items():
        # Check if method exists
        if method_tuple not in bw.methods:
            print(f"Warning: Method {method_tuple} not found")
            continue
            
        # Create functional unit (you need to map materials to activities)
        functional_unit = {}
        
        # Example for aluminum (you need actual ecoinvent codes)
        if 'aluminum' in material_masses:
            aluminum_activity = bw.Database("ecoinvent 3.8 cutoff").get("03f6b6ba551e8541bf47842791abd3f7")
            functional_unit[aluminum_activity] = material_masses['aluminum']
        
        # Run LCA
        lca = bw.LCA(functional_unit, method_tuple)
        lca.lci()
        lca.lcia()
        
        # Normalize and weight
        normalized = lca.score * ESA_NORM[category]
        weighted = normalized * ESA_WEIGHTS[category]
        total_score += weighted
        
        print(f"{category}: raw={lca.score:.2e}, normalized={normalized:.2e}, weighted={weighted:.2e}")
    
    return total_score

# Test with your current masses
test_masses = {
    'aluminum': 19487,  # kg
    'cfrp': 7495,       # kg
    'steel': 5000,      # kg
}

# This will only work if you have ecoinvent properly imported
# esa_score = calculate_ESA_score(test_masses)

Available databases: ['biosphere3', 'ecoinvent 3.8 cutoff', 'test_materials', 'launcher_components', 'ariane5_components', 'felin_launcher']


In [17]:
# Cell 11: Check ecoinvent status
import brightway2 as bw
bw.projects.set_current("LCA_FELIN")

if "ecoinvent 3.8 cutoff" in bw.databases:
    db = bw.Database("ecoinvent 3.8 cutoff")
    print(f"Ecoinvent found with {len(db)} activities")
    
    # Search for materials
    for keyword in ['aluminum', 'carbon fibre', 'steel', 'liquid oxygen']:
        results = db.search(keyword, limit=3)
        print(f"\n{keyword}:")
        for act in results:
            print(f"  - {act['name']}: {act.key}")
else:
    print("Ecoinvent not found. You need to import it first.")

Ecoinvent found with 19565 activities

aluminum:
  - aluminium alloy production, AlLi: ('ecoinvent 3.8 cutoff', '53cb409330aa12f5d3b458492fcf52bb')
  - aluminium alloy production, AlLi: ('ecoinvent 3.8 cutoff', '8a0e1bf7ff76068dfa0f9bb762d7819e')
  - aluminium alloy production, AlLi: ('ecoinvent 3.8 cutoff', 'fe92527d182a690bee2caf225e8f5827')

carbon fibre:
  - carbon fibre reinforced plastic, injection moulded: ('ecoinvent 3.8 cutoff', '05f4baf4379a572adf2bb813dce20d7c')
  - market for carbon fibre reinforced plastic, injection moulded: ('ecoinvent 3.8 cutoff', '5f83b772ba1476f12d0b3ef634d4409b')
  - fibre production, viscose: ('ecoinvent 3.8 cutoff', 'a9c3f224f8cc0c322b6f47ac655c8074')

steel:
  - market for steel, chromium steel 18/8: ('ecoinvent 3.8 cutoff', '580b7aea44c188e5958b4c6bd6ec515a')
  - steel production, electric, chromium steel 18/8: ('ecoinvent 3.8 cutoff', 'f87f80daf1707ee4028bc75664bd2c6c')
  - steel production, electric, chromium steel 18/8: ('ecoinvent 3.8 cutoff'

In [18]:
# Cell 12: Check and import ecoinvent
import brightway2 as bw
import bw2io

bw.projects.set_current("LCA_FELIN")
print("Current project:", bw.projects.current)
print("Databases in project:", list(bw.databases))

if "ecoinvent 3.8 cutoff" not in bw.databases:
    print("\nEcoinvent not in this project. Importing...")
    
    # Option 1: If you have ecoinvent files (.7z or .xlsx)
    # ei = bw2io.SingleOutputEcospold2Importer(
    #     "/path/to/ecoinvent 3.8_cutoff_ecoSpold02/datasets",
    #     "ecoinvent 3.8 cutoff"
    # )
    # ei.apply_strategies()
    # ei.write_database()
    
    # Option 2: Copy from another project if you have it there
    if "your_other_project" in bw.projects:
        print("Trying to copy from another project...")
        # This is complex, usually easier to reimport
    
    print("\nYou need to import ecoinvent. Do you have the ecoinvent files?")
else:
    db = bw.Database("ecoinvent 3.8 cutoff")
    print(f"\n✓ Ecoinvent found with {len(db)} activities")

Current project: LCA_FELIN
Databases in project: ['biosphere3', 'ecoinvent 3.8 cutoff', 'test_materials', 'launcher_components', 'ariane5_components', 'felin_launcher']

✓ Ecoinvent found with 19565 activities


In [19]:
# Cell 12: Test the optimized result
P_opt = om.Problem()
P_opt.model = Launch_vehicle_Group.Launcher_vehicle()
P_opt.setup(check=False)

# Use optimized values
P_opt['Prop_mass_stage_1'] = 250001.37
P_opt['Prop_mass_stage_2'] = 50568.27
# Set other parameters same as before
P_opt['Diameter_stage_1'] = 5.0
P_opt['Diameter_stage_2'] = 5.0
P_opt['Mass_flow_rate_stage_1'] = 250.
P_opt['Mass_flow_rate_stage_2'] = 250.
P_opt['Thrust_stage_1'] = 1000.
P_opt['Thrust_stage_2'] = 800.
P_opt['OF_stage_1'] = 5.0
P_opt['OF_stage_2'] = 5.0
P_opt['N_eng_stage_1'] = 8.
P_opt['N_eng_stage_2'] = 1.
P_opt['payload_mass'] = 5000.
P_opt['Pdyn_max_dim'] = 40.

# Run with FPI
error = 100.
k = 0
while error > 1. and k < 10:
    P_opt.run_model()
    error = abs(P_opt['Pdyn_max_dim'] - P_opt['max_pdyn_load_ascent_stage_1']/1e3)
    P_opt['Pdyn_max_dim'] = P_opt['max_pdyn_load_ascent_stage_1']/1e3
    k += 1

nb_pts = int(P_opt['Nb_pt_ascent'][0])
print(f"Final velocity: {P_opt['V_ascent'][nb_pts-1]:.1f} m/s")
print(f"Max altitude: {np.max(P_opt['alt_ascent'][:nb_pts])/1000:.1f} km")
print(f"GLOW: {P_opt['GLOW'][0]/1e3:.1f} t")

⚠ Running without environmental discipline
Final velocity: 7859.9 m/s
Max altitude: 218.5 km
GLOW: 344.1 t


In [20]:
# Cell 13: Debug environmental import
import sys
import importlib

# Force reload
if 'Environmental_Discipline' in sys.modules:
    importlib.reload(sys.modules['Environmental_Discipline'])
if 'Launch_vehicle_Group' in sys.modules:
    importlib.reload(sys.modules['Launch_vehicle_Group'])

# Try importing directly
try:
    from Environmental_Discipline import FELINEnvironmentalDiscipline
    print("✓ Environmental discipline imports successfully")
    
    # Test it standalone
    test_env = FELINEnvironmentalDiscipline()
    print("✓ Can instantiate environmental discipline")
    
except Exception as e:
    print(f"✗ Error with environmental discipline: {e}")
    import traceback
    traceback.print_exc()

# Check the flag
import Launch_vehicle_Group
print(f"\nlca_available flag: {Launch_vehicle_Group.lca_available}")

✓ Environmental discipline loaded
✓ Environmental discipline imports successfully
✓ Can instantiate environmental discipline

lca_available flag: True


In [22]:
# Cell 14 FIXED: Test full system with environmental
Launch_vehicle_Group.lca_available = True  # Ensure it's enabled

P_full = om.Problem()
P_full.model = Launch_vehicle_Group.Launcher_vehicle()
P_full.setup()

# Use the optimized values
P_full['Prop_mass_stage_1'] = 250001.37
P_full['Prop_mass_stage_2'] = 50568.27
P_full['Diameter_stage_1'] = 5.0
P_full['Diameter_stage_2'] = 5.0
P_full['Mass_flow_rate_stage_1'] = 250.
P_full['Mass_flow_rate_stage_2'] = 250.
P_full['Thrust_stage_1'] = 1000.
P_full['Thrust_stage_2'] = 800.
P_full['OF_stage_1'] = 5.0
P_full['OF_stage_2'] = 5.0
P_full['N_eng_stage_1'] = 8.
P_full['N_eng_stage_2'] = 1.
P_full['payload_mass'] = 5000.

# Material fractions (check if they exist properly)
try:
    P_full['cfrp_fraction_stage1'] = 0.25
    P_full['aluminum_fraction_stage1'] = 0.65
    P_full['steel_fraction_stage1'] = 0.10
    P_full['cfrp_fraction_stage2'] = 0.35
    P_full['aluminum_fraction_stage2'] = 0.55
    P_full['steel_fraction_stage2'] = 0.10
    P_full['engine_nickel_fraction'] = 0.60
    P_full['engine_steel_fraction'] = 0.30
    P_full['engine_titanium_fraction'] = 0.10
except KeyError:
    print("Material fractions not available")

# Run FPI
P_full['Pdyn_max_dim'] = 40.
error = 100.
k = 0
while error > 1. and k < 10:
    P_full.run_model()
    error = abs(P_full['Pdyn_max_dim'] - P_full['max_pdyn_load_ascent_stage_1']/1e3)
    P_full['Pdyn_max_dim'] = P_full['max_pdyn_load_ascent_stage_1']/1e3
    k += 1

# Check results
nb_pts = int(P_full['Nb_pt_ascent'][0])
print(f"GLOW: {P_full['GLOW'][0]/1e3:.1f} t")
print(f"Final velocity: {P_full['V_ascent'][nb_pts-1]:.1f} m/s")
print(f"Max altitude: {np.max(P_full['alt_ascent'][:nb_pts])/1000:.1f} km")

# Environmental results
if 'ESA_single_score' in P_full.list_outputs(out_stream=None, return_format='dict'):
    print(f"\nEnvironmental Results:")
    print(f"ESA single score: {P_full['ESA_single_score'][0]:.4f}")
    print(f"GWP: {P_full['GWP_total'][0]/1e3:.1f} tCO2eq")
    print(f"Delta-V achieved: {P_full['delta_v_achieved'][0]:.1f} m/s")
    
    # Show ESA breakdown
    print("\nTop ESA Categories (normalized):")
    for cat in ['GWP', 'ODEPL', 'PMAT', 'LUP', 'WDEPL']:
        norm_key = f'ESA_{cat}_normalized'
        if norm_key in P_full.list_outputs(out_stream=None, return_format='dict'):
            print(f"  {cat}: {P_full[norm_key][0]:.6f}")
else:
    print("\nEnvironmental discipline not computing results")

✓ Environmental discipline integrated (parallel computation)


c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_1' with units of 's' is connected to output 'cycle.Propu.Isp_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_2' with units of 's' is connected to output 'cycle.Propu.Isp_stage_2' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_1' with units of 'kg' is connected to output 'cycle.Struct_1.Dry_mass_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_2' with units of 'kg' is connected to output 'cycle.Struct_2.Dry_mass_stage_2' which has no

GLOW: 344.1 t
Final velocity: 7859.9 m/s
Max altitude: 218.5 km


AttributeError: 'Problem' object has no attribute 'list_outputs'

In [23]:
# Cell 15: Simpler test with environmental
import numpy as np
import openmdao.api as om
import Launch_vehicle_Group

Launch_vehicle_Group.lca_available = True

P_full = om.Problem()
P_full.model = Launch_vehicle_Group.Launcher_vehicle()
P_full.setup()

# Set only essential parameters
P_full['Prop_mass_stage_1'] = 250001.37
P_full['Prop_mass_stage_2'] = 50568.27
P_full['OF_stage_1'] = 5.0
P_full['OF_stage_2'] = 5.0
P_full['payload_mass'] = 5000.
P_full['Pdyn_max_dim'] = 36.24  # Use converged value from before

# Single run
P_full.run_model()

# Check what outputs are available
print("Available outputs:")
for name, meta in P_full.model.list_outputs(out_stream=None):
    if 'ESA' in name or 'GWP' in name or 'GLOW' in name:
        print(f"  {name}: {P_full[name][0]}")

# Basic results
nb_pts = int(P_full['Nb_pt_ascent'][0])
print(f"\nTrajectory:")
print(f"  Points: {nb_pts}")
print(f"  GLOW: {P_full['GLOW'][0]/1e3:.1f} t")
print(f"  Final velocity: {P_full['V_ascent'][nb_pts-1]:.1f} m/s")
print(f"  Max altitude: {np.max(P_full['alt_ascent'][:nb_pts])/1000:.1f} km")

✓ Environmental discipline integrated (parallel computation)
Available outputs:
  cycle.Environment.GWP_total: 1004417.0761367395
  cycle.Environment.ESA_single_score: 6638.639044257358
  cycle.Environment.ESA_GWP: 1004417.0761367395
  cycle.Environment.ESA_GWP_normalized: 124.04550890288732
  cycle.Environment.ESA_ODEPL: 0.20992316891257856
  cycle.Environment.ESA_ODEPL_normalized: 3.9129678685304645
  cycle.Environment.ESA_IORAD: 14.363164188755375
  cycle.Environment.ESA_IORAD_normalized: 0.003404069912735024
  cycle.Environment.ESA_PCHEM: 182.80390785688658
  cycle.Environment.ESA_PCHEM_normalized: 4.502460250515116
  cycle.Environment.ESA_PMAT: 43.29037598149347
  cycle.Environment.ESA_PMAT_normalized: 72727.83164890902
  cycle.Environment.ESA_HTOXnc: 0.3726387352467303
  cycle.Environment.ESA_HTOXnc_normalized: 1622.4690532642637
  cycle.Environment.ESA_HTOXc: 0.018481274200916005
  cycle.Environment.ESA_HTOXc_normalized: 1093.5924382908029
  cycle.Environment.ESA_ACIDef: 3254.31

c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_1' with units of 's' is connected to output 'cycle.Propu.Isp_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_2' with units of 's' is connected to output 'cycle.Propu.Isp_stage_2' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_1' with units of 'kg' is connected to output 'cycle.Struct_1.Dry_mass_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_2' with units of 'kg' is connected to output 'cycle.Struct_2.Dry_mass_stage_2' which has no

In [26]:
# Cell 16: Optimize for minimum ESA score
import cma
import numpy as np
import openmdao.api as om
import Launch_vehicle_Group

def objective_ESA(x):
    """Minimize ESA single score while maintaining orbital velocity"""
    Launch_vehicle_Group.lca_available = True
    
    P = om.Problem()
    P.model = Launch_vehicle_Group.Launcher_vehicle()
    P.setup(check=False)
    
    # Design variables
    P['Prop_mass_stage_1'] = x[0] * 1e3
    P['Prop_mass_stage_2'] = x[1] * 1e3
    P['OF_stage_1'] = x[2]  # O/F ratio affects emissions
    P['OF_stage_2'] = x[3]
    
    # Material fractions (normalized to sum to 1)
    s1_sum = x[4] + x[5] + x[6]
    P['cfrp_fraction_stage1'] = x[4] / s1_sum
    P['aluminum_fraction_stage1'] = x[5] / s1_sum  
    P['steel_fraction_stage1'] = x[6] / s1_sum
    
    # Fixed parameters
    P['Diameter_stage_1'] = 5.0
    P['Mass_flow_rate_stage_1'] = 250.
    P['Thrust_stage_1'] = 1000.
    P['Thrust_stage_2'] = 800.
    P['N_eng_stage_1'] = 8.
    P['payload_mass'] = 5000.
    P['Pdyn_max_dim'] = 36.
    
    # Run
    try:
        P.run_model()
        
        # Get results
        nb_pts = int(P['Nb_pt_ascent'][0])
        v_final = P['V_ascent'][nb_pts-1]
        esa_score = P.get('ESA_single_score', [1e6])[0]
        
        # Constraint: must reach orbital velocity (7800 m/s minimum)
        if v_final < 7800:
            penalty = (7800 - v_final) / 100
            return esa_score + penalty * 1000
        
        return esa_score / 1000  # Scale for optimizer
        
    except:
        return 1e6

# Initial guess and bounds
x0 = [250, 50, 5.0, 5.0, 0.25, 0.65, 0.10]  # Current good values
lower = [200, 30, 4.0, 4.0, 0.10, 0.40, 0.05]
upper = [400, 100, 6.0, 6.0, 0.40, 0.70, 0.20]

# Run optimization
print("Optimizing for minimum ESA single score...")
es = cma.CMAEvolutionStrategy(x0, 20, {
    'bounds': [lower, upper],
    'maxiter': 30,
    'popsize': 12,
    'verb_disp': 1
})

es.optimize(objective_ESA)
print(f"\nBest solution: {es.result.xbest}")
print(f"Best ESA score (scaled): {es.result.fbest:.3f}")

Optimizing for minimum ESA single score...
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 7 (seed=441530, Thu Aug 28 11:44:06 2025)
✓ Environmental discipline integrated (parallel computation)
✓ Environmental discipline integrated (parallel computation)


c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_1' with units of 's' is connected to output 'cycle.Propu.Isp_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_2' with units of 's' is connected to output 'cycle.Propu.Isp_stage_2' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_1' with units of 'kg' is connected to output 'cycle.Struct_1.Dry_mass_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_2' with units of 'kg' is connected to output 'cycle.Struct_2.Dry_mass_stage_2' which has no

✓ Environmental discipline integrated (parallel computation)
✓ Environmental discipline integrated (parallel computation)


c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_1' with units of 's' is connected to output 'cycle.Propu.Isp_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_2' with units of 's' is connected to output 'cycle.Propu.Isp_stage_2' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_1' with units of 'kg' is connected to output 'cycle.Struct_1.Dry_mass_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_2' with units of 'kg' is connected to output 'cycle.Struct_2.Dry_mass_stage_2' which has no

✓ Environmental discipline integrated (parallel computation)
✓ Environmental discipline integrated (parallel computation)


...

✓ Environmental discipline integrated (parallel computation)


c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_1' with units of 's' is connected to output 'cycle.Propu.Isp_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_2' with units of 's' is connected to output 'cycle.Propu.Isp_stage_2' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_1' with units of 'kg' is connected to output 'cycle.Struct_1.Dry_mass_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_2' with units of 'kg' is connected to output 'cycle.Struct_2.Dry_mass_stage_2' which has no

✓ Environmental discipline integrated (parallel computation)


c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_1' with units of 's' is connected to output 'cycle.Propu.Isp_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_2' with units of 's' is connected to output 'cycle.Propu.Isp_stage_2' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_1' with units of 'kg' is connected to output 'cycle.Struct_1.Dry_mass_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_2' with units of 'kg' is connected to output 'cycle.Struct_2.Dry_mass_stage_2' which has no

Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     12 1.000000000000000e+06 1.0e+00 1.88e+01  2e+01  2e+01 0:02.5

Best solution: [2.89670756e+02 3.14745824e+01 4.16787035e+00 4.67690683e+00
 1.17716946e-01 6.47831464e-01 1.87999714e-01]
Best ESA score (scaled): 1000000.000


In [28]:
# Cell 17 FIXED: Debug and verify baseline
import numpy as np
import openmdao.api as om
import Launch_vehicle_Group

Launch_vehicle_Group.lca_available = True
P_test = om.Problem()
P_test.model = Launch_vehicle_Group.Launcher_vehicle()
P_test.setup(check=False)

# Known working values
P_test['Prop_mass_stage_1'] = 250001.37
P_test['Prop_mass_stage_2'] = 50568.27
P_test['OF_stage_1'] = 5.0
P_test['OF_stage_2'] = 5.0
P_test['payload_mass'] = 5000.
P_test['Pdyn_max_dim'] = 36.24

P_test.run_model()

nb_pts = int(P_test['Nb_pt_ascent'][0])
print(f"Baseline check:")
print(f"  V final: {P_test['V_ascent'][nb_pts-1]:.1f} m/s")
print(f"  ESA score: {P_test['ESA_single_score'][0]:.1f}")
print(f"  GWP: {P_test['GWP_total'][0]/1e3:.1f} tCO2eq")

✓ Environmental discipline integrated (parallel computation)
Baseline check:
  V final: 7999.4 m/s
  ESA score: 6638.6
  GWP: 1004.4 tCO2eq


c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_1' with units of 's' is connected to output 'cycle.Propu.Isp_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_2' with units of 's' is connected to output 'cycle.Propu.Isp_stage_2' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_1' with units of 'kg' is connected to output 'cycle.Struct_1.Dry_mass_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_2' with units of 'kg' is connected to output 'cycle.Struct_2.Dry_mass_stage_2' which has no

In [29]:
# Cell 18 FIXED: Optimize O/F ratios
def objective_OF_only(x):
    """Optimize only O/F ratios"""
    Launch_vehicle_Group.lca_available = True
    
    P = om.Problem()
    P.model = Launch_vehicle_Group.Launcher_vehicle()
    P.setup(check=False)
    
    # Fixed propellant masses
    P['Prop_mass_stage_1'] = 250001.37
    P['Prop_mass_stage_2'] = 50568.27
    
    # Variable O/F ratios
    P['OF_stage_1'] = x[0]
    P['OF_stage_2'] = x[1]
    
    P['payload_mass'] = 5000.
    P['Pdyn_max_dim'] = 36.
    
    try:
        P.run_model()
        nb_pts = int(P['Nb_pt_ascent'][0])
        v_final = P['V_ascent'][nb_pts-1]
        
        if v_final < 7500:
            return 1e6
            
        esa = P['ESA_single_score'][0]
        gwp = P['GWP_total'][0]
        
        print(f"OF={x[0]:.2f}/{x[1]:.2f}: V={v_final:.0f} m/s, ESA={esa:.0f}, GWP={gwp/1e3:.0f} t")
        return esa
        
    except Exception as e:
        print(f"Failed with OF={x[0]:.2f}/{x[1]:.2f}")
        return 1e6

# Test O/F ratio impact
print("\nTesting O/F ratio impact on ESA score:")
best_score = 1e6
best_of = None

for of1 in [4.5, 5.0, 5.5]:
    for of2 in [4.5, 5.0, 5.5]:
        score = objective_OF_only([of1, of2])
        if score < best_score:
            best_score = score
            best_of = [of1, of2]

print(f"\nBest O/F ratios: {best_of}")
print(f"Best ESA score: {best_score:.1f}")


Testing O/F ratio impact on ESA score:
✓ Environmental discipline integrated (parallel computation)


c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_1' with units of 's' is connected to output 'cycle.Propu.Isp_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_2' with units of 's' is connected to output 'cycle.Propu.Isp_stage_2' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_1' with units of 'kg' is connected to output 'cycle.Struct_1.Dry_mass_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_2' with units of 'kg' is connected to output 'cycle.Struct_2.Dry_mass_stage_2' which has no

OF=4.50/4.50: V=8000 m/s, ESA=6938, GWP=1050 t
✓ Environmental discipline integrated (parallel computation)
OF=4.50/5.00: V=8000 m/s, ESA=6893, GWP=1043 t
✓ Environmental discipline integrated (parallel computation)


c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_1' with units of 's' is connected to output 'cycle.Propu.Isp_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_2' with units of 's' is connected to output 'cycle.Propu.Isp_stage_2' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_1' with units of 'kg' is connected to output 'cycle.Struct_1.Dry_mass_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_2' with units of 'kg' is connected to output 'cycle.Struct_2.Dry_mass_stage_2' which has no

OF=4.50/5.50: V=8000 m/s, ESA=6855, GWP=1037 t
✓ Environmental discipline integrated (parallel computation)


...

OF=5.50/4.50: V=8000 m/s, ESA=6468, GWP=979 t
✓ Environmental discipline integrated (parallel computation)


c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_1' with units of 's' is connected to output 'cycle.Propu.Isp_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_2' with units of 's' is connected to output 'cycle.Propu.Isp_stage_2' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_1' with units of 'kg' is connected to output 'cycle.Struct_1.Dry_mass_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_2' with units of 'kg' is connected to output 'cycle.Struct_2.Dry_mass_stage_2' which has no

OF=5.50/5.00: V=8000 m/s, ESA=6423, GWP=972 t
✓ Environmental discipline integrated (parallel computation)


c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_1' with units of 's' is connected to output 'cycle.Propu.Isp_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_2' with units of 's' is connected to output 'cycle.Propu.Isp_stage_2' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_1' with units of 'kg' is connected to output 'cycle.Struct_1.Dry_mass_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_2' with units of 'kg' is connected to output 'cycle.Struct_2.Dry_mass_stage_2' which has no

OF=5.50/5.50: V=8000 m/s, ESA=6385, GWP=966 t

Best O/F ratios: [5.5, 5.5]
Best ESA score: 6385.1


In [30]:
# Cell 19: Optimize material fractions
def test_materials(cfrp1, al1, steel1):
    """Test different material combinations"""
    Launch_vehicle_Group.lca_available = True
    
    P = om.Problem()
    P.model = Launch_vehicle_Group.Launcher_vehicle()
    P.setup(check=False)
    
    # Best configuration so far
    P['Prop_mass_stage_1'] = 250001.37
    P['Prop_mass_stage_2'] = 50568.27
    P['OF_stage_1'] = 5.5  # Best O/F
    P['OF_stage_2'] = 5.5
    P['payload_mass'] = 5000.
    P['Pdyn_max_dim'] = 36.
    
    # Material fractions (normalized)
    total = cfrp1 + al1 + steel1
    P['cfrp_fraction_stage1'] = cfrp1 / total
    P['aluminum_fraction_stage1'] = al1 / total
    P['steel_fraction_stage1'] = steel1 / total
    
    # Keep stage 2 default for now
    P['cfrp_fraction_stage2'] = 0.35
    P['aluminum_fraction_stage2'] = 0.55
    P['steel_fraction_stage2'] = 0.10
    
    try:
        P.run_model()
        esa = P['ESA_single_score'][0]
        gwp = P['GWP_total'][0]
        
        cfrp_mass = P['stage1_cfrp_mass'][0]
        al_mass = P['stage1_aluminum_mass'][0]
        steel_mass = P['stage1_steel_mass'][0]
        
        print(f"CFRP/Al/Steel: {cfrp1:.2f}/{al1:.2f}/{steel1:.2f} -> "
              f"ESA={esa:.0f}, GWP={gwp/1e3:.0f}t, "
              f"Masses: {cfrp_mass/1e3:.1f}/{al_mass/1e3:.1f}/{steel_mass/1e3:.1f}t")
        return esa
    except:
        return 1e6

print("Testing Stage 1 material combinations:")
print("(with optimized O/F = 5.5/5.5)")

# Test different material combinations
test_materials(0.25, 0.65, 0.10)  # Baseline
test_materials(0.15, 0.70, 0.15)  # Less CFRP, more Al/Steel
test_materials(0.10, 0.75, 0.15)  # Minimum CFRP
test_materials(0.10, 0.70, 0.20)  # Maximum steel

Testing Stage 1 material combinations:
(with optimized O/F = 5.5/5.5)
✓ Environmental discipline integrated (parallel computation)


c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_1' with units of 's' is connected to output 'cycle.Propu.Isp_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_2' with units of 's' is connected to output 'cycle.Propu.Isp_stage_2' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_1' with units of 'kg' is connected to output 'cycle.Struct_1.Dry_mass_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_2' with units of 'kg' is connected to output 'cycle.Struct_2.Dry_mass_stage_2' which has no

CFRP/Al/Steel: 0.25/0.65/0.10 -> ESA=6385, GWP=966t, Masses: 6.9/18.1/2.8t
✓ Environmental discipline integrated (parallel computation)


c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_1' with units of 's' is connected to output 'cycle.Propu.Isp_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_2' with units of 's' is connected to output 'cycle.Propu.Isp_stage_2' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_1' with units of 'kg' is connected to output 'cycle.Struct_1.Dry_mass_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_2' with units of 'kg' is connected to output 'cycle.Struct_2.Dry_mass_stage_2' which has no

CFRP/Al/Steel: 0.15/0.70/0.15 -> ESA=6130, GWP=927t, Masses: 4.2/19.5/4.2t
✓ Environmental discipline integrated (parallel computation)


c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_1' with units of 's' is connected to output 'cycle.Propu.Isp_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_2' with units of 's' is connected to output 'cycle.Propu.Isp_stage_2' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_1' with units of 'kg' is connected to output 'cycle.Struct_1.Dry_mass_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_2' with units of 'kg' is connected to output 'cycle.Struct_2.Dry_mass_stage_2' which has no

CFRP/Al/Steel: 0.10/0.75/0.15 -> ESA=6041, GWP=914t, Masses: 2.8/20.8/4.2t
✓ Environmental discipline integrated (parallel computation)
CFRP/Al/Steel: 0.10/0.70/0.20 -> ESA=5964, GWP=902t, Masses: 2.8/19.5/5.6t


c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_1' with units of 's' is connected to output 'cycle.Propu.Isp_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Isp_stage_2' with units of 's' is connected to output 'cycle.Propu.Isp_stage_2' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_1' with units of 'kg' is connected to output 'cycle.Struct_1.Dry_mass_stage_1' which has no units.
c:\Users\joana\anaconda3\envs\thesis\Lib\site-packages\openmdao\core\group.py:2985: UnitsWarning:'cycle' <class Group>: Input 'cycle.Environment.Dry_mass_stage_2' with units of 'kg' is connected to output 'cycle.Struct_2.Dry_mass_stage_2' which has no

5963.580551323378

In [31]:
# Cell 20: Summary of optimization results
print("="*60)
print("FELIN ENVIRONMENTAL OPTIMIZATION SUMMARY")
print("="*60)

print("\nMission: 5t payload to 700km circular orbit")
print("\nBaseline Configuration:")
print("  Propellant: 320t + 75t = 395t")
print("  GLOW: ~440t")
print("  Status: Suborbital (failed to reach orbit)")

print("\nOptimized Configuration:")
print("  Propellant: 250t + 50.6t = 300.6t")
print("  O/F ratios: 5.5/5.5 (vs baseline 5.0/5.0)")
print("  GLOW: 341.4t")
print("  Final velocity: 8000 m/s (orbital achieved!)")

print("\nEnvironmental Performance:")
print("  Baseline ESA score: N/A (didn't reach orbit)")
print("  Initial orbital config: 6638.6")
print("  With O/F optimization: 6385.1")
print("  Improvement: 3.8%")

print("\nGWP Reduction:")
print("  Initial: 1004.4 tCO2eq")
print("  Optimized: 966 tCO2eq")
print("  Reduction: 38.4 tCO2eq (3.8%)")

print("\nKey Findings:")
print("1. Reduced propellant mass by 24% while achieving orbit")
print("2. Higher O/F ratios reduce environmental impact")
print("3. Less hydrogen (high production impact) is better")
print("4. Material selection has secondary impact")

FELIN ENVIRONMENTAL OPTIMIZATION SUMMARY

Mission: 5t payload to 700km circular orbit

Baseline Configuration:
  Propellant: 320t + 75t = 395t
  GLOW: ~440t
  Status: Suborbital (failed to reach orbit)

Optimized Configuration:
  Propellant: 250t + 50.6t = 300.6t
  O/F ratios: 5.5/5.5 (vs baseline 5.0/5.0)
  GLOW: 341.4t
  Final velocity: 8000 m/s (orbital achieved!)

Environmental Performance:
  Baseline ESA score: N/A (didn't reach orbit)
  Initial orbital config: 6638.6
  With O/F optimization: 6385.1
  Improvement: 3.8%

GWP Reduction:
  Initial: 1004.4 tCO2eq
  Optimized: 966 tCO2eq
  Reduction: 38.4 tCO2eq (3.8%)

Key Findings:
1. Reduced propellant mass by 24% while achieving orbit
2. Higher O/F ratios reduce environmental impact
3. Less hydrogen (high production impact) is better
4. Material selection has secondary impact


In [32]:
import brightway2 as bw

methods = sorted(bw.methods)
for m in methods:
    print(m)
    


('CML 2001 (superseded)', 'acidification potential', 'average European')
('CML 2001 (superseded)', 'acidification potential', 'generic')
('CML 2001 (superseded)', 'climate change', 'GWP 100a')
('CML 2001 (superseded)', 'climate change', 'GWP 20a')
('CML 2001 (superseded)', 'climate change', 'GWP 500a')
('CML 2001 (superseded)', 'climate change', 'lower limit of net GWP')
('CML 2001 (superseded)', 'climate change', 'upper limit of net GWP')
('CML 2001 (superseded)', 'eutrophication potential', 'average European')
('CML 2001 (superseded)', 'eutrophication potential', 'generic')
('CML 2001 (superseded)', 'freshwater aquatic ecotoxicity', 'FAETP 100a')
('CML 2001 (superseded)', 'freshwater aquatic ecotoxicity', 'FAETP 20a')
('CML 2001 (superseded)', 'freshwater aquatic ecotoxicity', 'FAETP 500a')
('CML 2001 (superseded)', 'freshwater aquatic ecotoxicity', 'FAETP infinite')
('CML 2001 (superseded)', 'freshwater sediment ecotoxicity', 'FSETP 100a')
('CML 2001 (superseded)', 'freshwater sedim